# Ejecta model

TARDIS requires a model of the homologously expanding ejecta in order to run a simulation. The ejecta is divided in concentric 1 dimensional radial cells to use as computational units. Each cell contains information the velocity structure, abundances, density, and a radiation field.

The information of these properties are stored as classes and all of them inherit from a `BaseProperty` to facilitate the interaction. 
`BaseProperty` handles the initailization of the properties' `time` and initial time (`time_0`). If no `time_0` is provided, it will default to 0 seconds (except density which defaults to 1 second) and if no `time` is provided, it will default to `time_0`. `check_number_of_cells()` is used by the Generic model to check consistency.
The function `cgs.units()` is implemented in the children classes.
Any more properties added to the GenericModel should be built on this class. They must include a `name` attribute, a `cgs_units()` function to check and convert units, and should delegate time initalization to `BaseProperty`.            
##  Model grid
The first thing needed in order to have a useful ejecta model is a discreet spatial grid. 
What defines the spatial grid in TARDIS is the velocity grid. This dependance on velocity also entails a dependance with time.
### Velocity Class
For a basic grid inner and outer velocities are needed. Each concentric cell has an inner and outer velocity, and `inner_velocity` and `outer_velocity` are arrays composed of these velocities.
The velocity class stores velocities and velocities-related quantities (such as cell radius and volume). To start a Velocity class we need `inner_velocity` and an `outer_velocity` arrays, optional arguments are `time_0` which fallback to 0 seconds and `time` which is recomended to have singificant values for radii and volume.


In [1]:
from tardis.model._generic_model import *
from tardis.io.model_reader import read_csv_isotope_abundances

/home/jordi/conda/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
velocity_inner = [i for i in range(9000, 13000, 500)] * u.km / u.s
velocity_outer = [i for i in range(9500, 13500, 500)] * u.km / u.s
print(velocity_inner,'\n',velocity_outer)

[ 9000.  9500. 10000. 10500. 11000. 11500. 12000. 12500.] km / s 
 [ 9500. 10000. 10500. 11000. 11500. 12000. 12500. 13000.] km / s


In [3]:
velocity = Velocity(velocity_inner,velocity_outer)

no `time` provided, `time_0` will be used


Quantities are converted to CGS and can be easily accessed as attributes:

In [4]:
velocity.middle

<Quantity [9.250e+08, 9.750e+08, 1.025e+09, 1.075e+09, 1.125e+09,
           1.175e+09, 1.225e+09, 1.275e+09] cm / s>

By not providing a time for the class some quantities don't make sense:

In [5]:
velocity.inner_radius

<Quantity [0., 0., 0., 0., 0., 0., 0., 0.] cm>

`time` can be modified:

In [6]:
velocity.time = 1 * u.d
velocity.time

<Quantity 1. d>

Quantities stored in Velocity are:
 - Velocity.inner 
 - Velocity.outer
 - Velocity.middle
 - Velocity.inner_radius
 - Velocity.middle_radius
 - Velocity.outer_radius
 - Velocity.volume
 - Velocity.time_0
 - Velocity.time
 - Velocity.number_of_cells

The number of cells is defined by the length of the input velocity arrays

In [7]:
velocity.number_of_cells

8

### Density
Once the grid is defined, more properties can be added to the structure. 
The Density class can store mass density and electron density(optional).
Initial mass density is stored as `mass_0`.

In [8]:
mass_density_array = [
            6.2419e-10,
            6.1502e-10,
            6.0492e-10,
            5.9475e-10,
            5.8468e-10,
            5.7473e-10,
            5.6493e-10,
            5.5529e-10,
        ] * u.g / u.cm**3
density = Density(mass_density_array, time = 1 * u.d)

Initial mass density is stored as `mass_0`.

In [9]:
density.time

<Quantity 86400. s>

The ejecta is homologouly expanding, and therefore the value of the density will decrease proportionally to (time/time_0)**-3. This again highlights the importance of providing a `time` for the class.

In [10]:
density.mass

<Quantity [9.67777630e-25, 9.53559970e-25, 9.37900389e-25, 9.22132276e-25,
           9.06519208e-25, 8.91092195e-25, 8.75897750e-25, 8.60951377e-25] g / cm3>

Quantities stored in Density are:
 - Density.mass_0
 - Density.mass
 - Density.electron
 - Density.time_0
 - Density.time
 - Density.number_of_cells

### Abundance
Fractional mass abundances are stored in the Abundance class and contain both element and isotopic abundance.
The easiest way to start an abundance class is using the tardis configuration file. Is this case (and to illustrate its capabilities) we will extract data from a file that contains elements and isotopes.

In [11]:
index, abundance, isotope_abundance = read_csv_isotope_abundances('examples/tardis_model_abundance_class.csv')

!cat ./examples/tardis_model_abundance_class.csv
#change .elemental to .element
#Show 'isotope' label in df
#how to display element/isotope

Index C O Mg Si Ni56 Ni58
0 0 0 0 0.6 0.4 0
1 0 0 0 0.1 0.5 0.4
2 0 0 0 0.3 0 0.7
3 0 0.2 0.8 0 0 0
4 0 0.3 0.7 0 0 0
5 0 0.2 0.8 0 0 0
6 0 0.2 0.8 0 0 0
7 0 0.2 0.8 0 0 0
8 0.5 0.5 0 0 0 0


To start an Abundance class you need and abundance and isotope abundances dataframes, along with time_0 and time, which are required for decay calculations.


In [12]:
abundances = Abundances(abundance, isotope_abundance, time = 1 * u.d)

`element` contains element abundances decayed by `time` - `time_0` if no time is provided, `time` = `time_0` and isotopes will not decay. Also note that this adds isotopes of the same element, for example ni56 and Ni58 end up adding to 0.84. 

In [13]:
abundances.elemental

element,C,O,Mg,Si,Fe,Co,Ni
cell,,,,,,,
0,0.0,0.0,0.0,0.1,0.000246,0.053667,0.846087
1,0.0,0.0,0.0,0.3,0.000000,0.000000,0.700000
2,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
3,0.0,0.3,0.7,0.0,0.000000,0.000000,0.000000
4,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
5,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
6,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
7,0.5,0.5,0.0,0.0,0.000000,0.000000,0.000000


For a more granular approach to isotopes, access `isotope`

In [14]:
abundances.isotope

,Fe56,Co56,Ni56,Ni58
cell,,,,
0,0.000246,0.053667,0.446085,0.4
1,0.000000,0.000000,0.000000,0.7
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0


To check initial abundances, acces `isotope_0` and `element_0`.

### Radiation Field
RadiationField class stores `radiative_temperature` and `dilution_factor` for every cell. The class can be instantiated with either quantity, also takes `time_0` and `time` like the others, but has no use for it.

In [15]:
radiative_temperature = [
        7000,
        7000,
        7000,
        7000,
        7000,
        7000,
        7000,
        7000,
    ] * u.K
dilution_factor = [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
radiation_field = RadiationField(radiative_temperature, dilution_factor)

no `time` provided, `time_0` will be used


`radiative_temperature` and `dilution_factor` are accessed as attributes.

In [16]:
radiation_field.radiative_temperature

<Quantity [7000., 7000., 7000., 7000., 7000., 7000., 7000., 7000.] K>

In [17]:
radiation_field.dilution_factor

[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]

## Generic[Ejecta]Model
A more flexible implementation of the TARDIS ejecta model is the Generic[Ejecta]Model. 
GenericModel is the easiest way to bind and syncronyze properties. A GenericModel is started with Property arguments and a time argument.
We can use the properties defined above to start a GenericModel.


In [18]:
ejecta = GenericModel(velocity,density,abundances,radiation_field,time = 1 * u.d)

Model time set to 86400.0 s or 1.0 d


Note that the time used in the model will transfer to all properties, and a particular property's `time` is grater that the one specified in the `GenericModel`, the greater time will be used. If a property has a different `number of cells`, the GenericModel will not start.

All properties can be accessed as normal attributes, and will always be stored under the same name, regardless of what the initial property class was called. For example, if out Velocity instance was called just 'vel', in the `GenericModel` it is still `GenericModel.velocity.inner`

In [19]:
ejecta.velocity.inner

<Quantity [9.00e+08, 9.50e+08, 1.00e+09, 1.05e+09, 1.10e+09, 1.15e+09,
           1.20e+09, 1.25e+09] cm / s>

In [20]:
ejecta.abundance.elemental

element,C,O,Mg,Si,Fe,Co,Ni
cell,,,,,,,
0,0.0,0.0,0.0,0.1,0.000246,0.053667,0.846087
1,0.0,0.0,0.0,0.3,0.000000,0.000000,0.700000
2,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
3,0.0,0.3,0.7,0.0,0.000000,0.000000,0.000000
4,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
5,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
6,0.0,0.2,0.8,0.0,0.000000,0.000000,0.000000
7,0.5,0.5,0.0,0.0,0.000000,0.000000,0.000000


## Ejecta Dataframe

Customs calculations can be difficult, for a more classic aproach to the data stored by the model, the method `to_dataframe()` generates a dataframe.